In [50]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
import yaml
import os
import pandas as pd
import numpy as np
import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
import aequitas.plot as ap

In [22]:
datadir = os.path.join(os.getcwd(),'data/donors-choose/single-model-audit')

In [5]:
with open(os.path.join(datadir,'best_precision.yaml')) as file:
    metadata = yaml.load(file, Loader=yaml.FullLoader)

# Model with Highest Precision at top 50

In [6]:
metadata

{'val_predictions': '',
 'test_predictions': 'predictions_c598fbe93f4c218ac7d325fb478598f1.csv.gz',
 'model_uuid': 'a04e2eedd9c5ff18bcf77e84ae9db561',
 'model_classpath': 'sklearn.ensemble.RandomForestClassifier',
 'model_hyperparameters': None,
 'training_set': {'matrix_uuid': '',
  'num_rows': 0,
  'num_cols': 114,
  'label_col': 'quickstart_label'},
 'eval_params': {'target_pp': {1000: {'global_threshold': 0.527624878013593}}}}

In [7]:
testdf = pd.read_csv(os.path.join(datadir,metadata['test_predictions']), compression='gzip')

In [8]:
testdf.shape

(17677, 4)

# Thresholding


In [9]:
testdf.head()

,entity_id,as_of_date,quickstart_label,predict_proba
0,185730,1354320000000,0.0,0.308565
1,185729,1354320000000,0.0,0.371010
2,185738,1354320000000,0.0,0.438470
3,185737,1354320000000,0.0,0.470824
4,185736,1354320000000,0.0,0.248637


In [11]:
global_threshold = metadata['eval_params']['target_pp'][1000]['global_threshold'] - 0.0001
global_threshold

0.527524878013593

In [12]:
testdf['score'] = testdf['predict_proba'].apply(lambda x: 1 if x >= global_threshold else 0 )

In [13]:
testdf.score.value_counts()

0    16675
1     1002
Name: score, dtype: int64

In [17]:
indices = np.argsort(testdf['score'])[::-1]
threshold = testdf['score'][indices][1000 - 1]

In [18]:
indices

17676     6356
17675      720
17674     5449
17673    16915
17672    11925
         ...  
4        11626
3        11625
2        11624
1        11623
0            0
Name: score, Length: 17677, dtype: int64

In [19]:
testdf['label_value'] = testdf['quickstart_label']


In [51]:
attr_df = pd.read_csv(os.path.join(datadir,'donors_choose_protected.csv.gz'), compression='gzip')
attr_df.tail(10)

,entity_id,as_of_date,metro_type,grade_level,poverty_level,teacher_sex
165608,168445,1359590400000,urban,9-12,high,female
165609,168446,1359590400000,suburban,PreK-2,high,female
165610,168447,1359590400000,rural,PreK-2,high,female
165611,168448,1359590400000,urban,PreK-2,high,female
165612,168449,1359590400000,urban,6-8,high,female
165613,168451,1359590400000,urban,PreK-2,high,female
165614,168452,1359590400000,suburban,3-5,high,female
165615,168453,1359590400000,unknown,PreK-2,high,female
165616,168430,1359590400000,rural,PreK-2,high,female
165617,168476,1359590400000,urban,9-12,high,female


In [52]:
attr_df['poverty_']

(165618, 6)

In [53]:
df = pd.merge(testdf[['entity_id','as_of_date','score','label_value']], attr_df, how='left', on=['entity_id','as_of_date'], left_index=True, right_index=False, sort=True, copy=True)

In [27]:
df.tail()

,entity_id,as_of_date,score,label_value,metro_type,grade_level,poverty_level,teacher_sex
148273,186059,1354320000000,0,0.0,rural,PreK-2,high,female
148274,186060,1354320000000,0,0.0,suburban,3-5,high,female
148275,186061,1354320000000,0,0.0,unknown,PreK-2,low_moderate,female
148276,186062,1354320000000,0,0.0,urban,9-12,high,female
148278,186063,1354320000000,0,0.0,urban,PreK-2,high,female


## Aequitas

In [29]:
g = Group()
xtab, _ = g.get_crosstabs(df[['score','label_value','poverty_level','metro_type', 'teacher_sex']].copy())

model_id, score_thresholds 0 {'rank_abs': [1002]}


In [30]:
absolute_metrics = g.list_absolute_metrics(xtab)

In [31]:
xtab[[col for col in xtab.columns if col not in absolute_metrics]]

,model_id,score_threshold,k,attribute_name,attribute_value,pp,pn,fp,fn,tn,tp,group_label_pos,group_label_neg,group_size,total_entities
0,0,binary 0/1,1002,poverty_level,high,562,13866,200,4473,9393,362,4835,9593,14428,17677
1,0,binary 0/1,1002,poverty_level,low_moderate,440,2809,249,944,1865,191,1135,2114,3249,17677
2,0,binary 0/1,1002,metro_type,rural,368,1610,173,630,980,195,825,1153,1978,17677
3,0,binary 0/1,1002,metro_type,suburban,259,4269,120,1429,2840,139,1568,2960,4528,17677
4,0,binary 0/1,1002,metro_type,unknown,183,1873,84,666,1207,99,765,1291,2056,17677
5,0,binary 0/1,1002,metro_type,urban,192,8923,72,2692,6231,120,2812,6303,9115,17677
6,0,binary 0/1,1002,teacher_sex,female,953,14139,425,4619,9520,528,5147,9945,15092,17677
7,0,binary 0/1,1002,teacher_sex,male,49,2535,24,798,1737,25,823,1761,2584,17677
8,0,binary 0/1,1002,teacher_sex,unknown,0,1,0,0,1,0,0,1,1,17677


In [32]:
xtab[['attribute_name', 'attribute_value'] + absolute_metrics]

,attribute_name,attribute_value,tpr,tnr,for,fdr,fpr,fnr,npv,precision,ppr,pprev,prev
0,poverty_level,high,0.074871,0.979151,0.322588,0.355872,0.020849,0.925129,0.677412,0.644128,0.560878,0.038952,0.335112
1,poverty_level,low_moderate,0.168282,0.882214,0.336063,0.565909,0.117786,0.831718,0.663937,0.434091,0.439122,0.135426,0.349338
2,metro_type,rural,0.236364,0.849957,0.391304,0.470109,0.150043,0.763636,0.608696,0.529891,0.367265,0.186047,0.417088
3,metro_type,suburban,0.088648,0.959459,0.334739,0.463320,0.040541,0.911352,0.665261,0.536680,0.258483,0.057200,0.346290
4,metro_type,unknown,0.129412,0.934934,0.355579,0.459016,0.065066,0.870588,0.644421,0.540984,0.182635,0.089008,0.372082
5,metro_type,urban,0.042674,0.988577,0.301692,0.375000,0.011423,0.957326,0.698308,0.625000,0.191617,0.021064,0.308502
6,teacher_sex,female,0.102584,0.957265,0.326685,0.445960,0.042735,0.897416,0.673315,0.554040,0.951098,0.063146,0.341042
7,teacher_sex,male,0.030377,0.986371,0.314793,0.489796,0.013629,0.969623,0.685207,0.510204,0.048902,0.018963,0.318498
8,teacher_sex,unknown,NaN,1.000000,0.000000,NaN,0.000000,NaN,1.000000,NaN,0.000000,0.000000,0.000000


In [45]:
b = Bias()
bdf = b.get_disparity_predefined_groups(xtab, original_df=df, ref_groups_dict={'poverty_level':'high', 'metro_type':'urban', 'teacher_sex':'male'})

get_disparity_predefined_group()


In [40]:
bdf = b.get_disparity_min_metric(xtab, original_df=df)

get_disparity_min_metric()


In [46]:
metrics = ['tpr']

In [47]:
ap.disparities(bdf, metrics, 'poverty_level', fairness_threshold = 1.10)

alt.LayerChart(...)

In [48]:
ap.disparities(bdf, metrics, 'metro_type', fairness_threshold = 1.10)

alt.LayerChart(...)

In [49]:
ap.disparities(bdf, metrics, 'teacher_sex', fairness_threshold = 1.10)

alt.LayerChart(...)